In [1]:
#Imports

import os
import requests
import datetime as dt
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json
import numpy as np
import yfinance as yf
from pandas_datareader import data as pdr
import seaborn as sns
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv()

True

In [3]:
# #Download Datasets using yfinance
# #gld = pdr.get_data_yahoo('GLD')
# #gld = yf.download('gld', '2016-01-01', '2020-05-31')
# #gld = yf.download('gld', '2000-01-01', '2020-05-31')
# gld = yf.download('BTC-USD', '2016-01-01', '2022-01-06')
# #gld.drop(columns = ["Open","High","Low","Adj Close","Volume"], inplace=True)
# day = np.arange(1, len(gld) + 1)
# gld['day'] = day
# gld.drop(columns=['Adj Close', 'Volume'], inplace = True)
# gld = gld[['Close']]
# gld.head(-1)

In [7]:
#Import data using pandas_datareader

#Import Traditional Asset Data - Indicies covering Stocks, Bonds & T-bills

assetList = ['^GSPC', '^DJI', '^TNX', 'BND']
endDate = dt.datetime.now()
startDate = '2010-01-01'

#def getData_tdnl(assets, start, end):
assetData = pdr.DataReader(assetList, 'yahoo', startDate, endDate)
assetData = assetData['Close']
assetData.dropna(inplace=True)
    #return assetData
round(assetData.head(-1),2)


Symbols,^GSPC,^DJI,^TNX,BND
Date,,,,
2010-01-04,1132.99,10583.96,3.84,78.68
2010-01-05,1136.52,10572.02,3.76,78.91
2010-01-06,1137.14,10573.68,3.81,78.88
2010-01-07,1141.69,10606.86,3.82,78.82
2010-01-08,1144.98,10618.19,3.81,78.90
...,...,...,...,...
2021-12-31,4766.18,36338.30,1.51,84.75
2022-01-03,4796.56,36585.06,1.63,84.18
2022-01-04,4793.54,36799.65,1.67,84.17


In [9]:
#computing daily returns on the dataset

#assetReturns = assetData/assetData.shift(1)
#assetReturns.dropna(inplace=True)

assetReturns = assetData.pct_change()
#assetReturns.dropna(inplace=True)

#log returns
#logReturns = np.log(assetReturns)
#logReturns.dropna(inplace=True)

#mean returns
#meanReturns = assetReturns.mean()

#computing annualised return    
#annual_meanReturns = (1+assetReturns.mean())**252 - 1

#computing annual covariance    

#covMatrix = logReturns.cov()
#sns.heatmap(covMatrix)
covMatrix = assetReturns.cov()

assetReturns
#meanReturns
#logReturns

Symbols,^GSPC,^DJI,^TNX,BND
Date,,,,
2010-01-04,NaN,NaN,NaN,NaN
2010-01-05,0.003116,-0.001128,-0.022390,0.002923
2010-01-06,0.000546,0.000157,0.014115,-0.000380
2010-01-07,0.004001,0.003138,0.003676,-0.000761
2010-01-08,0.002882,0.001068,-0.003663,0.001015
...,...,...,...,...
2022-01-03,0.006374,0.006791,0.076720,-0.006726
2022-01-04,-0.000630,0.005866,0.024570,-0.000119
2022-01-05,-0.019393,-0.010667,0.022182,-0.003445


In [10]:
covMatrix

Symbols,^GSPC,^DJI,^TNX,BND
Symbols,,,,
^GSPC,0.000117,0.000113,0.000134,-0.000003
^DJI,0.000113,0.000115,0.000138,-0.000003
^TNX,0.000134,0.000138,0.000918,-0.000054
BND,-0.000003,-0.000003,-0.000054,0.000007


In [11]:
#return assetReturns
    #return annual_meanReturns, covMatrix
    
#getData_tdnl(
    #assets=assetList,
    #start=startDate, 
    #end=endDate)

In [20]:
#Creating random portfolios

#- How many assests to include in each portfolio
n_assets = 4
#-- How many portfolios to generate
n_portfolios = 1000

#-- Initialize empty list to store mean-variance pairs for plotting

weight=np.zeros((n_portfolios,n_assets))
expectedReturns=np.zeros(n_portfolios)
expectedVolatility=np.zeros(n_portfolios)
sharpeRatio=np.zeros(n_portfolios)
meanassetReturn = assetReturns.mean()
sigma = assetReturns.cov()

np.random.seed(500)
#-- Loop through and generate lots of random portfolios
for k in range(n_portfolios):
    #- Choose assets randomly without replacement
    #assets = np.random.choice(list(assetReturns.columns), n_assets, replace=False)
    #- Choose weights randomly
    weights = np.random.rand(n_assets)
    #- Ensure weights sum to 1
    weights = weights/np.sum(weights)
    weight[k,:]=weights
    #expected Returns
    expectedReturns[k] = np.sum(meanassetReturn * weights)
    #expected volatility
    expectedVolatility[k] = np.sqrt(np.dot(weights.T,np.dot(sigma,weights)) )
    #Sharpe Ratio
    sharpeRatio[k] = expectedReturns[k]/expectedVolatility[k]
sharpeRatio

array([0.02250438, 0.01913249, 0.01810566, 0.02651727, 0.01792212,
       0.02445299, 0.03225439, 0.01830499, 0.03811983, 0.01488411,
       0.02579033, 0.02514373, 0.03517576, 0.04175551, 0.02633229,
       0.02021239, 0.04304452, 0.03021375, 0.02324647, 0.02959482,
       0.0263394 , 0.03782326, 0.02291556, 0.02065529, 0.03353625,
       0.02857656, 0.02329614, 0.02663888, 0.02775328, 0.0276194 ,
       0.01839012, 0.02341686, 0.03360785, 0.02644683, 0.028822  ,
       0.0325218 , 0.01649962, 0.02302791, 0.02474565, 0.02051347,
       0.04014921, 0.03227776, 0.0454993 , 0.04215542, 0.01811809,
       0.02706966, 0.02838757, 0.02611571, 0.02068367, 0.01318517,
       0.03265819, 0.02319369, 0.02941027, 0.02820142, 0.0275407 ,
       0.02057675, 0.01579527, 0.04549353, 0.04673396, 0.02568178,
       0.02771426, 0.02515488, 0.04607353, 0.03502996, 0.02713571,
       0.02246973, 0.03034519, 0.02478912, 0.04580695, 0.02628334,
       0.03438399, 0.01731339, 0.01749091, 0.04333135, 0.02266

In [21]:
#Efficient Frontier
from scipy.optimize import minimize

def negativeSR(weights):
    w = np.array(weights)
    r = np.sum(meanassetReturn*weights)
    v = np.sqrt(np.dot(weights.T,np.dot(sigma,weights)) )
    SR = r/v
    
    return -1*SR

def checkSumToOne(weights):
    return np.sum(weights)-1
    
w0 = [0.25,0.25,0.25,0.25]
#w0 = [0.20,0.20,0.20,0.20,0.20]
bounds = ((0,1),(0,1),(0,1),(0,1))
#bounds = ((0,1),(0,1),(0,1),(0,1),(0,1))
constraints = ({'type':'eq','fun':checkSumToOne})
w_opt = minimize(negativeSR,w0,method='SLSQP',bounds=bounds,constraints=constraints)
w_opt

     fun: -0.05074504175434458
     jac: array([-0.00016665,  0.00823026,  0.02834074,  0.00014486])
 message: 'Optimization terminated successfully'
    nfev: 56
     nit: 11
    njev: 11
  status: 0
 success: True
       x: array([0.47059066, 0.        , 0.        , 0.52940934])

In [22]:
maxIndex = sharpeRatio.argmax()
weight[maxIndex,:]

weights*100

array([ 6.44638592, 22.49546303, 14.88031999, 56.17783106])

In [ ]:
returns = np.linspace(-0.04,0.10,50)
volatility_opt = []
def minimizemyVolatility(weights):
    w = np.array(weights)
    v = np.sqrt(np.dot(weights.T,np.dot(sigma,weights)) )
    return v
def getReturn(weights):
    w = np.array(weights)
    R = np.sum(meanlogReturn*weights)
    return R
for R in returns:
    #find best volatility
    constraints = ({'type':'eq','fun':checkSumToOne},
                   {'type':'eq','fun':lambda weights: getReturn(weights)-R})
    opt = minimize(minimizemyVolatility,w0,method='SLSQP',bounds=bounds,constraints=constraints)
    #save optimal volatility
    volatility_opt.append(opt['fun'])


In [ ]:
plt.figure(figsize=(12,7))
plt.ylabel('Expected Returns')
plt.xlabel('Expected Volatility')
plt.scatter(expectedVolatility,expectedReturns,c=sharpeRatio)
plt.colorbar(label = 'Sharpe Ratio')
plt.scatter(expectedVolatility[maxIndex],expectedReturns[maxIndex],c='red')
plt.plot(volatility_opt,returns,'--')
plt.show()

In [ ]:
# Crypto



In [ ]:
#Import data using pandas_datareader

#Import Popular Crypto Data

# def getData_crypto(crypto, start, end):
#     cryptoData = pdr.get_data_yahoo(crypto, start=startDate, end=endDate)
#     cryptoData = cryptoData['Close']
#     return cryptoData

# cryptoList = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'DOT-USD', 'SOL-USD', 'ADA-USD','DOGE-USD','XRP-USD','SHIB-USD']

# endDate = dt.datetime.now()
# startDate = '2010-01-01'

# print(getData(crypto=cryptoList, start=startDate, end=endDate))

cryptoList = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'DOT-USD', 'SOL-USD', 'ADA-USD','DOGE-USD','XRP-USD']
endDate = dt.datetime.now()
startDate = '2010-01-01'

#def getData_tdnl(assets, start, end):
cryptoData = pdr.DataReader(cryptoList, 'yahoo', startDate, endDate)
cryptoData = cryptoData['Close']
cryptoData.dropna(inplace=True)
    #return assetData
cryptoData.head(-1)

In [ ]:
# Creating a Crypto Index with popular Cryptos based on MarketCap

In [ ]:
#computing daily returns on the dataset
cryptoReturns = cryptoData/cryptoData.shift(1)
#assetReturns.dropna(inplace=True)
   
#log returns
cryptologReturns = np.log(cryptoReturns)
cryptoReturns.dropna(inplace=True)

#mean returns
#meanReturns = assetReturns.mean()

#computing annualised return    
#annual_meanReturns = (1+assetReturns.mean())**252 - 1

#computing annual covariance    
#covMatrix = logReturns.cov()
#sns.heatmap(covMatrix)

assetReturns
#meanReturns
cryptologReturns
cryptoReturns

In [ ]:
#Creating random crypto portfolios

#- How many assests to include in each portfolio
n_crypto = 8
#-- How many portfolios to generate
n_Cportfolios = 1000

#-- Initialize empty list to store mean-variance pairs for plotting

Cweight=np.zeros((n_Cportfolios,n_crypto))
CexpectedReturns=np.zeros(n_Cportfolios)
CexpectedVolatility=np.zeros(n_Cportfolios)
CsharpeRatio=np.zeros(n_Cportfolios)
CmeanlogReturn = cryptologReturns.mean()
Csigma = cryptologReturns.cov()
np.random.seed(75)
#-- Loop through and generate lots of random portfolios
for k in range(n_Cportfolios):
    #- Choose assets randomly without replacement
    #assets = np.random.choice(list(assetReturns.columns), n_assets, replace=False)
    #- Choose weights randomly
    Cweights = np.random.rand(n_crypto)
    #- Ensure weights sum to 1
    Cweights = Cweights/np.sum(Cweights)
    Cweight[k,:]=Cweights
    #expected Returns
    CexpectedReturns[k] = np.sum(CmeanlogReturn * Cweights)
    #expected volatility
    CexpectedVolatility[k] = np.sqrt(np.dot(Cweights.T,np.dot(Csigma,Cweights)) )
    #Sharpe Ratio
    CsharpeRatio[k] = CexpectedReturns[k]/CexpectedVolatility[k]